# Fuzzy ART com classes ordenadas

## Importando bibliotecas

In [60]:
%pip install artlib

Note: you may need to restart the kernel to use updated packages.


In [61]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.datasets import USPS
from artlib import FuzzyART, FuzzyARTMAP, FusionART
from sklearn.metrics import classification_report, adjusted_rand_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import resample
from pyclustertend import vat
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Métricas

In [62]:
def generate_acc_matrix_fuzzyART(num_tasks, X_train_sorted, y_train_sorted, X_test_sorted, y_test_sorted, images):
    train_subsets = []
    test_subsets = []

    acc_matrix = [[0 for _ in range(num_tasks)] for _ in range(num_tasks)]

    fuzzy_art_model = FuzzyART(rho=0.1, alpha=0.0, beta=1.0)

    if(images):
        lower_bounds = np.zeros(16*16)
        upper_bounds = np.full(16*16, 255.0)
        fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)
    else:
        X_combined = np.concatenate([X_train_subset, X_test_subset], axis=0)
        lower_bound, upper_bound = fuzzy_art_model.find_data_bounds(X_combined)
        fuzzy_art_model.set_data_bounds(lower_bound, upper_bound)

    for i in range(num_tasks):
        for j in range(num_tasks):
            # Classes até a i-ésima (inclusive)
            train_classes = torch.arange(0, i + 1)

            # Máscara de seleção para treino: todas as classes <= i
            mask_train = torch.isin(y_train_sorted, train_classes)
            X_train_subset = X_train_sorted[mask_train]
            y_train_subset = y_train_sorted[mask_train]

            # Máscara de seleção para teste: apenas a classe j
            mask_test = (y_test_sorted == j)
            X_test_subset = X_test_sorted[mask_test]
            y_test_subset = y_test_sorted[mask_test]

            # Armazena os subconjuntos (opcional)
            train_subsets.append((X_train_subset, y_train_subset))
            test_subsets.append((X_test_subset, y_test_subset))

            if len(X_train_subset) == 0 or len(X_test_subset) == 0:
                print("Problema: subconjunto de dados vazio")
                acc_matrix[i][j] = 0.0
                continue
        
            train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train_subset)
            test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test_subset)

            fuzzy_art_model.partial_fit(train_X_fuzzy_art)
            fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

            acc_matrix[i][j] = adjusted_rand_score(y_test_subset, fuzzy_art_predictions)

    return acc_matrix

In [63]:
def average_accuracy(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks+1)/2
  acc_sum = 0

  for i in range(num_tasks):
    for j in range(i):
      acc_sum += acc_matrix[i][j]

  return (acc_sum / denominator)

def backward_transfer(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks-1)/2
  acc_sum = 0

  for i in range(2, num_tasks):
    for j in range(i-1):
      acc_sum += (acc_matrix[i][j] - acc_matrix[j][j])

  return (acc_sum / denominator)

def forward_transfer(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks-1)/2
  acc_sum = 0
  j = 0

  for i in range(j-1):
    for j in range(2, num_tasks):
      acc_sum = acc_sum + acc_matrix[i][j]

  return (acc_sum / denominator)

## Baixando e ordenando dados

In [64]:
train_data = USPS(root='./USPS/', train=True, download=True)
test_data = USPS(root='./USPS/', train=False, download=True)

X_train = train_data.data
y_train = torch.tensor(train_data.targets, dtype=torch.long)

X_test = test_data.data
y_test = torch.tensor(test_data.targets, dtype=torch.long)

print(X_train.shape)

# as imagens do conjunto são 16x16  
n_dim = 16 * 16

#flatten images
X_train = X_train.reshape(-1, n_dim)
X_test = X_test.reshape(-1, n_dim)

# Ordenando dados
sorted_indices_train = torch.argsort(y_train)
X_train_sorted = X_train[sorted_indices_train]
y_train_sorted = y_train[sorted_indices_train]

sorted_indices_test = torch.argsort(y_test)
X_test_sorted = X_test[sorted_indices_test]
y_test_sorted = y_test[sorted_indices_test]

(7291, 16, 16)


In [65]:
print(y_train_sorted.unique())
print(y_test_sorted.unique())

y_train_sorted

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


tensor([0, 0, 0,  ..., 9, 9, 9])

In [66]:
classes = y_train.unique()
print(classes)
num_tasks = len(classes)
print(num_tasks)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
10


## Fuzzy ART

Aprendizado não supervionado


In [80]:
fuzzy_art_model = FuzzyART(rho=0.7, alpha=0.0, beta=1.0)

# Estabelecendo bounds
lower_bounds = np.zeros(n_dim) ## Isso foi feito com base no exemplo que está no github da biblioteca
upper_bounds = np.full(n_dim, 255.0)

fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)

# Normalização e complement coding
train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train)
test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test)

In [81]:
fuzzy_art_model.partial_fit(train_X_fuzzy_art)
fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

In [82]:
ARI = adjusted_rand_score(y_test,fuzzy_art_predictions)
print(f'O ARI do modelo FuzzyART foi: {ARI}')
num_clusters = fuzzy_art_model.n_clusters
print(f"O número de clusters foi: {num_clusters}")

O ARI do modelo FuzzyART foi: 0.03313772875665044
O número de clusters foi: 1773


In [70]:
acc_matrix = generate_acc_matrix_fuzzyART(num_tasks, X_train_sorted, y_train_sorted, 
                                            X_test_sorted, y_test_sorted, 1)





In [73]:
acc_matrix_df = pd.DataFrame(acc_matrix)

print(acc_matrix_df)

     0    1    2    3    4    5    6    7    8    9
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
3  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [72]:
print(f"Forward transfer: {forward_transfer(num_tasks, acc_matrix):.6f}")
print(f"Backward transfer: {backward_transfer(num_tasks, acc_matrix):.6f}")
print(f"Average Accuracy: {average_accuracy(num_tasks, acc_matrix):.6f}")


Forward transfer: 0.000000
Backward transfer: -0.266667
Average Accuracy: 0.218182


<font color="red" oi>